In [ ]:
import polars as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import glob
import os
import json
from tqdm import tqdm

# ------------------------------------------------------------------
# 1. 파일 필터링 및 데이터 로드 (202412 ~ 202511)
# ------------------------------------------------------------------
base_path = "../food/"
files_202412 = glob.glob(os.path.join(base_path, "*202412*.parquet"))
files_2025 = glob.glob(os.path.join(base_path, "*2025*.parquet"))
# 2025년 1월~11월 파일만 필터링
files_2025_filtered = [f for f in files_2025 if any(f"2025{str(m).zfill(2)}" in f for m in range(1, 12))]
all_files = files_202412 + files_2025_filtered

print(f"📦 읽어올 파일 개수: {len(all_files)}개")

# 데이터 스캔 및 500만 건 샘플링
raw_df = (
    pl.scan_parquet(all_files)
    .select(["admi_cty_no", "card_tpbuz_nm_2", "hour", "sex", "age", "day", "cnt"])
    .filter((pl.col("cnt") > 0) & (pl.col("age").is_between(2, 6)))
    .collect(engine="streaming")
)

sampling_fraction = 5000000 / len(raw_df)
# 전체 데이터 대비 500만 건 비율로 샘플링
train_df_raw = raw_df.sample(fraction=sampling_fraction, seed=42)
print(f"✅ 샘플링 완료: {len(train_df_raw):,} 건 (전체의 {sampling_fraction*100:.1f}%)")

# ------------------------------------------------------------------
# 2. 매핑 및 데이터 타입 최적화 (메모리 절약)
# ------------------------------------------------------------------
def get_mapping(df, col_name):
    unique_vals = df[col_name].cast(pl.String).unique().to_list()
    mapping = {val: i for i, val in enumerate(unique_vals)}
    return mapping, len(unique_vals)

area_map, n_areas = get_mapping(train_df_raw, "admi_cty_no")
biz_map, n_classes = get_mapping(train_df_raw, "card_tpbuz_nm_2")
sex_map, n_sex = get_mapping(train_df_raw, "sex")
inv_biz_map = {v: k for k, v in biz_map.items()}

# 데이터 타입 경량화 및 인덱스 변환
train_df = train_df_raw.with_columns([
    (pl.col("cnt").cast(pl.Float32) + 1).log().alias("log_cnt"),
    pl.col("admi_cty_no").cast(pl.String).replace(area_map).cast(pl.Int32),
    pl.col("card_tpbuz_nm_2").cast(pl.String).replace(biz_map).cast(pl.Int32),
    pl.col("sex").cast(pl.String).replace(sex_map).cast(pl.Int8),
    (pl.col("age").cast(pl.Int8) - 2).alias("age_idx"),
    pl.col("day").cast(pl.Int8),
    pl.col("hour").cast(pl.Float32)
])

# ------------------------------------------------------------------
# 3. 모델 및 데이터셋 정의
# ------------------------------------------------------------------
class FoodRecommender(nn.Module):
    def __init__(self, n_areas, n_classes, n_sex):
        super(FoodRecommender, self).__init__()
        self.area_emb = nn.Embedding(n_areas, 16)
        self.sex_emb = nn.Embedding(n_sex, 2)
        self.age_emb = nn.Embedding(5, 4)
        self.day_emb = nn.Embedding(8, 4)
        self.fc = nn.Sequential(
            nn.Linear(27, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, n_classes)
        )

    def forward(self, area, hour, sex, age, day):
        x = torch.cat([
            self.area_emb(area), self.sex_emb(sex),
            self.age_emb(age), self.day_emb(day),
            hour.unsqueeze(1)
        ], dim=1)
        return self.fc(x)

class FoodDataset(Dataset):
    def __init__(self, df):
        self.area = df["admi_cty_no"].to_numpy()
        self.hour = df["hour"].to_numpy()
        self.sex = df["sex"].to_numpy()
        self.age = df["age_idx"].to_numpy()
        self.day = df["day"].to_numpy()
        self.label = df["card_tpbuz_nm_2"].to_numpy()
        self.weights = df["log_cnt"].to_numpy()

    def __len__(self): return len(self.label)
    def __getitem__(self, idx):
        return (torch.tensor(self.area[idx], dtype=torch.long), 
                torch.tensor(self.hour[idx], dtype=torch.float32),
                torch.tensor(self.sex[idx], dtype=torch.long), 
                torch.tensor(self.age[idx], dtype=torch.long),
                torch.tensor(self.day[idx], dtype=torch.long), 
                torch.tensor(self.label[idx], dtype=torch.long),
                torch.tensor(self.weights[idx], dtype=torch.float32))

# ------------------------------------------------------------------
# 4. 학습 실행 (A100 GPU 최적화)
# ------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FoodRecommender(n_areas, n_classes, n_sex).to(device)
loader = DataLoader(FoodDataset(train_df), batch_size=32768, shuffle=True, num_workers=2, pin_memory=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(reduction='none')

print("🚀 학습 시작...")
model.train()
for epoch in range(5):
    loss_sum = 0
    for area, hour, sex, age, day, label, weight in tqdm(loader, desc=f"Epoch {epoch+1}"):
        area, hour, sex, age, day, label, weight = \
            area.to(device), hour.to(device), sex.to(device), age.to(device), \
            day.to(device), label.to(device), weight.to(device)
        
        optimizer.zero_grad()
        output = model(area, hour, sex, age, day)
        loss = (criterion(output, label) * weight).mean()
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    print(f"✅ Epoch {epoch+1} Loss: {loss_sum/len(loader):.4f}")

# ------------------------------------------------------------------
# 5. 모든 상황별 JSON 결과 저장
# ------------------------------------------------------------------
def get_hour_idx(real_hour):
    if 0 <= real_hour < 7: return 0
    elif 7 <= real_hour < 9: return 1
    elif 9 <= real_hour < 11: return 2
    elif 11 <= real_hour < 13: return 3
    elif 13 <= real_hour < 15: return 4
    elif 15 <= real_hour < 17: return 5
    elif 17 <= real_hour < 19: return 6
    elif 19 <= real_hour < 21: return 7
    elif 21 <= real_hour < 23: return 8
    elif 23 <= real_hour < 24: return 9
    else: return 10

print("📂 JSON 결과 생성 중...")
recommendation_map = {}
model.eval()
with torch.no_grad():
    for age in [2, 3, 4, 5, 6]:
        for sex in ['M', 'F']:
            for day in range(1, 8):
                for hour in range(24):
                    s_idx = sex_map[sex]
                    ag_idx = age - 2
                    d_idx = day - 1
                    h_idx = get_hour_idx(hour)
                    
                    area_t = torch.tensor([0], device=device)
                    sex_t = torch.tensor([s_idx], device=device)
                    age_t = torch.tensor([ag_idx], device=device)
                    day_t = torch.tensor([d_idx], device=device)
                    hour_t = torch.tensor([float(h_idx)], device=device)
                    
                    outputs = model(area_t, hour_t, sex_t, age_t, day_t)
                    probs = F.softmax(outputs, dim=1)
                    all_probs, all_indices = torch.topk(probs, n_classes)
                    
                    ranked_list = []
                    for i in range(n_classes):
                        idx = all_indices[0][i].item()
                        ranked_list.append({
                            "rank": i + 1,
                            "category": inv_biz_map.get(idx, "Unknown"),
                            "probability": f"{round(all_probs[0][i].item() * 100, 2)}%"
                        })
                    recommendation_map[f"{age}_{sex}_{day}_{hour}"] = ranked_list

with open("final_recommendation_5m.json", "w", encoding="utf-8") as f:
    json.dump(recommendation_map, f, ensure_ascii=False, indent=4)

print("✨ 모든 작업 완료! 'final_recommendation_5m.json' 파일을 확인하세요.")

In [ ]:
# ------------------------------------------------------------------
# 6. 모델 저장하기
# ------------------------------------------------------------------
# 모델의 가중치(State Dict)를 저장합니다.
model_save_path = "food_recommender_v1.pth"
torch.save(model.state_dict(), model_save_path)

# 매핑 사전(biz_map 등)도 나중에 업종명을 찾기 위해 저장해두는 것이 좋습니다.
mapping_data = {
    "area_map": area_map,
    "biz_map": biz_map,
    "sex_map": sex_map,
    "inv_biz_map": inv_biz_map
}
with open("category_mappings.json", "w", encoding="utf-8") as f:
    json.dump(mapping_data, f, ensure_ascii=False, indent=4)

print(f"💾 모델과 매핑 사전이 저장되었습니다: {model_save_path}, category_mappings.json")


# ------------------------------------------------------------------
# 7. 나중에 모델 다시 불러오기 (예시)
# ------------------------------------------------------------------
def load_trained_model(model_path, n_areas, n_classes, n_sex):
    # 1. 동일한 구조의 모델 객체를 먼저 생성
    loaded_model = FoodRecommender(n_areas, n_classes, n_sex).to(device)
    
    # 2. 저장된 가중치를 모델에 입힘
    loaded_model.load_state_dict(torch.load(model_path))
    
    # 3. 평가 모드로 전환
    loaded_model.eval()
    return loaded_model

# 실행 예시:
# model = load_trained_model("food_recommender_v1.pth", n_areas, n_classes, n_sex)

In [ ]:
from google.colab import files
files.download("food_recommender_v1.pth")
files.download("category_mappings.json")